# Execution in Google Colab
You can run this Notebook in Google Colab. Google account required but no local installation, 100% browser based.

http://colab.research.google.com/github/davidnewschool/sound-delay/blob/main/colab.ipynb

## Distance Calculation

### Analyze the Delay in a video

In [ ]:
!git clone https://github.com/davidnewschool/sound-delay.git

In [ ]:
# Change directory to cloned git repository and show the files.
%cd sound-delay
!ls 

In [ ]:
!pip install -r requirements.txt
!pip install plotly
!pip install -U kaleido

In [ ]:
# remove the # on the next line if you run it local and want the mathplotlib window as a popup to interact with. Does not work on Google Colab.
# %matplotlib tk

# give the input video file, for exampe: example/video.mp4
%run plot_delay.py

### Experimental Visual Analysis

In [ ]:
import plotly.graph_objects as go

In [ ]:
# Create a figure
fig = go.Figure()

# Plot the two signals
fig.add_trace(go.Scatter(x=time_audio, y=loudness, mode='lines', name='loudness', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=time_video, y=red_intensity, mode='lines', name='red intensity', line=dict(color='red')))

# Update the layout, axes properties, and other attributes
fig.update_layout(
    xaxis_title="Time [seconds]",
    yaxis_title="Amplitude",
    yaxis_showticklabels=False,  # Hide y-axis ticks as in the original plot
    xaxis_range=[time_min, time_max],
    yaxis_range=[amp_min, amp_max],
    showlegend=True,
    title="Comparison of Loudness and Red Intensity over Time",
    xaxis=dict(
        title="Time [seconds]",
        rangeslider=dict(visible=True),
        type='linear'
    )  # This line adds the range slider
)

# Save the plot to the same path/name as the input video
output_image_path = video_path[:-3] + 'png'
# fig.write_image(output_image_path)  # Requires plotly to be installed with the "orca" extra: pip install plotly[orca]

# Display the plot
fig.show()


In [ ]:
# Update the layout
fig.update_layout(
    xaxis=dict(
        ticklen=10,  # Length of major ticks
        showgrid=True,  # Gridlines
    )
)

# Update x-axis for minor ticks representing each frame within a second when zoomed in
fig.update_xaxes(
    minor_tickmode="linear",
    minor_tick0=0,
    minor_dtick=1/frame_rate,
    minor_ticklen=0,  # Length of minor ticks
    minor_showgrid=True,
    minor_nticks=frame_rate * total_seconds  # Maximum number of minor ticks
)

# Display the updated plot
fig.show()


In [ ]:

# ---- BOOM ----

# 1. Compute the derivative of the red_intensity
red_intensity_derivative = np.diff(red_intensity) / np.diff(time_video)
time_video_deriv = time_video[:-1]  # Adjusted time axis for derivative

# Normalize the derivative so it's in the same amplitude range as the original signals
red_intensity_derivative_norm = (red_intensity_derivative - np.min(red_intensity_derivative)) / (np.max(red_intensity_derivative) - np.min(red_intensity_derivative))
red_intensity_derivative_norm = red_intensity_derivative_norm * (np.max(red_intensity) - np.min(red_intensity)) + np.min(red_intensity)

# 2. Find the spike in the derivative.
first_spike_deriv_red_index = np.argmax(red_intensity_derivative_norm)

# 3. After detecting the spike, find the first position where the derivative starts decreasing.
first_decreasing_point_after_spike = np.argmax(red_intensity_derivative_norm[first_spike_deriv_red_index:] < red_intensity_derivative_norm[first_spike_deriv_red_index])
slowdown_time = time_video_deriv[first_spike_deriv_red_index + first_decreasing_point_after_spike]

# ---- SOUND ----

# Adjust loudness and time_audio data for calculations to start from slowdown_time
start_idx_audio = np.where(time_audio >= slowdown_time)[0][0]
adjusted_time_audio = time_audio[start_idx_audio:]
adjusted_loudness = loudness[start_idx_audio:]

# 1. Compute the derivative of the adjusted loudness signal
loudness_derivative = np.diff(adjusted_loudness) / np.diff(adjusted_time_audio)

# 2. Determine threshold for detecting a spike in adjusted loudness
threshold = 3 * np.std(loudness_derivative)

# 3. Detect the first significant spike
first_spike_index = np.argmax(loudness_derivative > threshold)

# Get the time and value corresponding to the first spike in adjusted loudness
first_spike_time = adjusted_time_audio[first_spike_index]
first_spike_value = adjusted_loudness[first_spike_index]

# Calculate the standard deviation till the point of the max adjusted loudness
std_dev_loudness = np.std(adjusted_loudness[:first_spike_index])

# ---- PLOTTING AND ANNOTATING ----

fig = go.Figure()

# Plot original signals
fig.add_trace(go.Scatter(x=time_audio, y=loudness, mode='lines', name='Loudness', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=time_video, y=red_intensity, mode='lines', name='Red Intensity', line=dict(color='red')))
fig.add_trace(go.Scatter(x=time_video_deriv, y=red_intensity_derivative_norm, mode='lines', name='Normalized Red Intensity Derivative', line=dict(color='rgb(255,100,100)')))

# Annotation for the point where derivative starts decreasing
fig.add_annotation(
    x=slowdown_time,
    y=red_intensity_derivative_norm[first_spike_deriv_red_index + first_decreasing_point_after_spike],
    text=f'Derivative starts decreasing at {slowdown_time:.2f} s',
    showarrow=True,
    arrowhead=2
)

# Annotate the first spike for loudness
loudness_ay_offset = -std_dev_loudness * 5  # Offset adjusted based on std dev
fig.add_annotation(
    x=first_spike_time,
    y=first_spike_value,
    text=f'First Loudness Spike at {first_spike_time:.2f} s',
    showarrow=True,
    arrowhead=2,
    ax=0,
    ay=loudness_ay_offset
)

# Adding the range slider to the x-axis
fig.update_layout(
    xaxis=dict(
        rangeslider=dict(visible=True),
    )
)

# Display the plot
fig.show()


In [ ]:
def compute_derivative(data, time):
    """Compute the derivative and return the adjusted time."""
    derivative = np.diff(data) / np.diff(time)
    adjusted_time = time[:-1]
    return derivative, adjusted_time

def normalize_data(data):
    """Normalize data between 0 and 1 based on its range."""
    return (data - np.min(data)) / (np.max(data) - np.min(data))

# ---- BOOM ----

# Derive red_intensity to detect rapid changes which may correspond to visual anomalies (e.g., flash from an explosion).
red_intensity_derivative, time_video_deriv = compute_derivative(red_intensity, time_video)

# Normalize derivative to align its amplitude range with the original signal, enhancing visualization.
red_intensity_derivative_norm = normalize_data(red_intensity_derivative)
red_intensity_derivative_norm *= (np.max(red_intensity) - np.min(red_intensity)) + np.min(red_intensity)

# Identify the spike in the derivative, which likely indicates the start of the explosion.
first_spike_deriv_red_index = np.argmax(red_intensity_derivative_norm)

# Detect the point where the increase in intensity begins to slow down post-explosion.
first_decreasing_point_after_spike = np.argmax(red_intensity_derivative_norm[first_spike_deriv_red_index:] < red_intensity_derivative_norm[first_spike_deriv_red_index])
slowdown_time = time_video_deriv[first_spike_deriv_red_index + first_decreasing_point_after_spike]

# ---- SOUND ----

# Adjust the audio data to focus on the timeframe after the visual anomaly was detected.
start_idx_audio = np.where(time_audio >= slowdown_time)[0][0]
adjusted_time_audio, adjusted_loudness = time_audio[start_idx_audio:], loudness[start_idx_audio:]

# Derive the loudness to detect rapid changes in sound intensity.
loudness_derivative, _ = compute_derivative(adjusted_loudness, adjusted_time_audio)

# Define a threshold to detect the significant spike in loudness which likely corresponds to the sound from the explosion.
threshold = 3 * np.std(loudness_derivative)
first_spike_index = np.argmax(loudness_derivative > threshold)
first_spike_time, first_spike_value = adjusted_time_audio[first_spike_index], adjusted_loudness[first_spike_index]

# ---- PLOTTING AND ANNOTATING ----

# Visualize the original and derived signals for in-depth analysis.
fig = go.Figure(data=[
    go.Scatter(x=time_audio, y=loudness, mode='lines', name='Loudness', line=dict(color='blue')),
    go.Scatter(x=time_video, y=red_intensity, mode='lines', name='Red Intensity', line=dict(color='red')),
])

# Annotations to highlight key points of interest in the plot.
fig.add_annotation(x=slowdown_time, y=red_intensity_derivative_norm[first_spike_deriv_red_index + first_decreasing_point_after_spike], text=f'Derivative starts decreasing at {slowdown_time:.2f} s', showarrow=True, arrowhead=2)
fig.add_annotation(x=first_spike_time, y=first_spike_value, text=f'First Loudness Spike at {first_spike_time:.2f} s', showarrow=True, arrowhead=2, ax=0, ay=-5 * np.std(adjusted_loudness[:first_spike_index]))

# Adding the range slider facilitates detailed examination of specific time intervals.
fig.update_layout(xaxis=dict(rangeslider=dict(visible=True)))

# Add a rectangular shape to highlight the region
fig.add_shape(
    type="rect",
    xref="x",
    yref="paper",  # relative to the entire height of the plot
    x0=slowdown_time,
    x1=first_spike_time,
    y0=0,
    y1=1,
    fillcolor="grey",
    opacity=0.3,
    layer="below",  # place the shape below the traces
    line_width=0,
)


# Display the visual analysis.
fig.show()


In [ ]:
%run distance.py

## Quick Helper for handling video

### Download a mp4 file from a URL 
You can enter any URL (that links directly to a mp4 file) to download into the file drive of this running colab. Files will not be stored longterm and be deleted after you stop the runtime (or timeout)

Use Online Services like TwitterVideoDownloader.com or ttvdl.com (TikTok) to get a .mp4 link.

In [ ]:
import requests
from IPython.display import display, HTML

# Ask user for the URL
url = input("Please enter the URL of the MP4 file: ")

# Define a suitable filename based on the URL
filename = url.split('/')[-1]  # This will take the last part of the URL as the filename. 

response = requests.get(url)
with open(filename, 'wb') as f:
    f.write(response.content)

# Display a success message in the notebook
display(HTML(f"<span style='color: green;'>File downloaded successfully as <b>{filename}</b></span>"))

### Trimming video
This Python script enables you to trim a video by specifying the start and end times, creating a new video clip containing only the desired segment. You can use this script to extract specific portions of a video for further editing or sharing.

How to Use
Input Video File: The script will prompt you to enter the path of the MP4 video file you want to trim. Please provide the full file path, including the file extension (e.g: example/video.mp4).

Video Duration: After loading the video, the script will display the total duration of the video in seconds. This information helps you determine the range for trimming.

Specify Trimming Times: Enter the start and end times (in seconds) for the portion of the video you want to keep. The script will cut the video from the specified start time to the specified end time.

Output Video File: The trimmed video will be saved with a "-cut" suffix appended to the original filename. For example, if the original file was named "video.mp4," the trimmed video will be named "video-cut.mp4."

In [ ]:
from moviepy.video.io.VideoFileClip import VideoFileClip

In [ ]:
# Ask the user for the path of the MP4 file
video_path = input("Please enter the path of the MP4 file: ")

# Load the video clip
video_clip = VideoFileClip(video_path)

# Give the user information about the duration of the clip
video_duration = video_clip.duration
print(f"The video duration is {video_duration:.2f} seconds.")

# Define the start and end times for trimming (in seconds)
start_time = float(input("Please enter the start time where you want to cut: "))  # Start time of the trimmed portion
end_time = float(input("Please enter the end time where you want to cut: "))  # End time of the trimmed portion

# Trim the video clip
trimmed_clip = video_clip.subclip(start_time, end_time)

# Generate the output path with a "-cut" suffix
output_path = video_path.replace(".mp4", "-cut.mp4")

# Save the trimmed video with audio
trimmed_clip.write_videofile(output_path, codec="libx264")

# Close the original video clip
video_clip.close()


### Video Cropping
This Python script allows you to crop a video by specifying the percentage of the frame to cut from the top, bottom, left, and right sides. You can use this script to customize the framing of a video, removing unwanted portions to focus on specific content.

How to Use
Input Video File: The script will prompt you to enter the path of the MP4 video file you want to edit. Provide the full file path, including the file extension (e.g: example/video.mp4).

Percentage to Cut: You will be asked to specify the percentage (0-100) of each side (top, bottom, left, and right) that you want to cut. Higher percentages will result in more cropping, while lower percentages will retain more of the original frame.

Output Video File: The edited video will be saved with the specified cropping percentages appended to the filename. For example, if you entered 10% for top, 5% for bottom, 15% for left, and 20% for right, the output file would be named something like original-video-edit_10_5_15_20.mp4.

In [ ]:
from moviepy.video.io.VideoFileClip import VideoFileClip
import moviepy.video.fx.all as vfx

In [ ]:
# Input video file path
input_file_path = input("Please enter the path of the MP4 file: ")

# Load the video clip
video_clip = VideoFileClip(input_file_path)

# Get the dimensions of the video frame
frame_width, frame_height = video_clip.size

# Ask the user for the percentage to cut from each side
top_percentage = float(input("Enter the percentage to cut from the top (0-100): "))
bottom_percentage = float(input("Enter the percentage to cut from the bottom (0-100): "))
left_percentage = float(input("Enter the percentage to cut from the left (0-100): "))
right_percentage = float(input("Enter the percentage to cut from the right (0-100): "))

# Output video file path with percentages
output_file_path = input_file_path.replace(".mp4", f"-edit_{top_percentage}_{bottom_percentage}_{left_percentage}_{right_percentage}.mp4")

# Calculate the pixel values to cut
top_cut = int(frame_height * (top_percentage / 100))
bottom_cut = int(frame_height * (bottom_percentage / 100))
left_cut = int(frame_width * (left_percentage / 100))
right_cut = int(frame_width * (right_percentage / 100))

# Crop the video clip
cropped_clip = video_clip.crop(y1=top_cut, y2=frame_height - bottom_cut, x1=left_cut, x2=frame_width - right_cut)

# Write the edited video to the output file
cropped_clip.write_videofile(output_file_path, codec="libx264")

print("Video editing complete. Saved as", output_file_path)
